In [1]:
from flask import Flask, render_template, request ,jsonify
from flask_cors import CORS
import speech_recognition as sr
from pydub import AudioSegment
import os
import shutil


from keras.models import model_from_json
import nbimporter
import ChatBot_modelisation

import warnings
warnings.filterwarnings('ignore')

In [2]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
app = Flask(__name__)
CORS(app)

@app.post("/predict")
def predict ():
    text=request.get_json().get("message")
    response=ChatBot_modelisation.get_answer(text,model)
    message={"answer": response}
    return jsonify(message)


@app.post("/predict-audio")
def predict_audio():
    audio_file = request.files['audio']


  # Save the uploaded file
    temp_path = "temp" + os.path.splitext(audio_file.filename)[1]
    audio_file.save(temp_path)

    # Convert the audio file to a supported format (WAV)
    converted_path = "audio.wav"
    AudioSegment.from_file(temp_path).export(converted_path, format="wav")


    # Reconnaissance vocale
    r = sr.Recognizer()
    try:

        with sr.AudioFile(converted_path) as source:
            audio_data = r.record(source)
        text = r.recognize_google(audio_data, language="ar-AR")
        reponse= "لقد قلت : "+text+'\n'+ChatBot_modelisation.get_answer(text,model)

        response = {'answer': reponse}
        return jsonify(response)
    
    except:
        error_message = "لايمكنني سماعك جيدا، اعد المحاولة من فضلك"
        return jsonify({'answer': error_message})
    


if __name__=="__main__":
    app.run(debug=False)
    
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/May/2023 17:37:04] "OPTIONS /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 172ms/step


127.0.0.1 - - [18/May/2023 17:37:07] "POST /predict HTTP/1.1" 200 -
